In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv("WIPRO.csv")

In [3]:
data.head(2)

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2000-01-03,WIPRO,EQ,2522.4,2724.00,2724.20,2724.00,2724.20,2724.20,2724.17,1599,4.355942e+11,NaN,NaN,NaN
1,2000-01-04,WIPRO,EQ,2724.2,2942.15,2942.15,2942.15,2942.15,2942.15,2942.15,4392,1.292192e+12,NaN,NaN,NaN


In [4]:
data.drop(["Symbol","Series","Turnover","Trades","Deliverable Volume","%Deliverble"],axis=1,inplace=True)

In [5]:
data.drop("Date",axis=1,inplace=True)

In [6]:
target=data["Open"]
data.drop("Open",axis=1,inplace=True)

In [7]:
data.head(3)

,Prev Close,High,Low,Last,Close,VWAP,Volume
0,2522.40,2724.20,2724.00,2724.20,2724.20,2724.17,1599
1,2724.20,2942.15,2942.15,2942.15,2942.15,2942.15,4392
2,2942.15,3177.55,2715.00,3000.00,2990.10,3063.86,132297


In [8]:
data.shape

(5306, 7)

In [9]:
train,test,y_train,y_test=train_test_split(data,target,test_size=0.3)

In [10]:
y_train.shape

(3714,)

In [26]:
l=train[0:2].values

In [27]:
l

array([[2.7055000e+02, 2.7320000e+02, 2.6715000e+02, 2.6825000e+02,
        2.6885000e+02, 2.7034000e+02, 1.3990387e+07],
       [2.5475000e+02, 2.6000000e+02, 2.5380000e+02, 2.5990000e+02,
        2.5930000e+02, 2.5860000e+02, 3.4962620e+06]])

In [12]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
High_train

array([[0.03269351],
       [0.03341674],
       [0.08748007],
       ...,
       [0.11331602],
       [0.01213139],
       [0.03997867]])

In [35]:
def norma(data,tdata):
  prev=MinMaxScaler()
  prev_close=np.array(data)
  tprev_close=np.array(tdata)
  prev.fit(prev_close.reshape(-1,1))
  prev_train=prev.transform(prev_close.reshape(-1,1))
  prev_test=prev.transform(tprev_close.reshape(-1,1))
  return list(prev_train),list(prev_test)

In [36]:
train_prevclose,test_prevclose=norma(train["Prev Close"],test["Prev Close"])
train_high,test_high=norma(train["High"],test["High"])
train_low,test_low=norma(train["Low"],test["Low"])
train_last,test_last=norma(train["Last"],test["Last"])
train_close,test_close=norma(train["Close"],test["Close"])
train_VWAP,test_VWAP=norma(train["VWAP"],test["VWAP"])
train_volume,test_volume=norma(train["Volume"],test["Volume"])

In [69]:
train_norm=pd.DataFrame({"prevclose":train_prevclose,
                         "high":train_high,
                         "low":train_low,
                         "last":train_last
                         })

In [68]:
test_norm=pd.DataFrame({"prevclose":test_prevclose,
                         "high":test_high,
                         "low":test_low,
                         "last":test_last
                         })

In [41]:
def moving_window(data,end_index,start_index,target):
  data_set=[]
  target_set=[]
  kl=end_index-start_index
  print("the kl is",kl)
  i=0
  while i<3694:
    k=i
    l=i
   
   
    data_set.append(data[k:k+20].values)
    target_set.append(target[l+20])
    i=i+1
  return data_set,target_set

In [ ]:
len(train)

3714

In [71]:
jkl=y_train.values

In [ ]:
jkl[3713]

763.0

In [72]:
z,u=moving_window(train_norm,len(train),20,jkl)

the kl is 3694


In [ ]:
len(z)

3694

In [73]:
z=np.asarray(z).astype(np.float32)

In [74]:
z.shape

(3694, 20, 4)

In [75]:
u=np.asarray(u).astype(np.float32)

In [53]:
u.shape

(3694,)

In [54]:
import tensorflow as tf

In [55]:
### Create the Stacked LSTM model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [79]:
model=Sequential()
model.add(LSTM(20,return_sequences=True,input_shape=(20,4)))
model.add(LSTM(20,return_sequences=True))
model.add(LSTM(20))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [80]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 20, 20)            2000      
                                                                 
 lstm_10 (LSTM)              (None, 20, 20)            3280      
                                                                 
 lstm_11 (LSTM)              (None, 20)                3280      
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 8,581
Trainable params: 8,581
Non-trainable params: 0
_________________________________________________________________


In [81]:
model.fit(z,u,epochs=100,batch_size=64,verbose=1)


Epoch 1/100
58/58 [==============================] - 4s 7ms/step - loss: 1194214.5000
Epoch 2/100
58/58 [==============================] - 0s 7ms/step - loss: 1188025.2500
Epoch 3/100
58/58 [==============================] - 0s 7ms/step - loss: 1185416.2500
Epoch 4/100
58/58 [==============================] - 0s 6ms/step - loss: 1183273.5000
Epoch 5/100
58/58 [==============================] - 0s 7ms/step - loss: 1181257.3750
Epoch 6/100
58/58 [==============================] - 0s 6ms/step - loss: 1179311.5000
Epoch 7/100
58/58 [==============================] - 0s 6ms/step - loss: 1177404.0000
Epoch 8/100
58/58 [==============================] - 0s 6ms/step - loss: 1175528.7500
Epoch 9/100
58/58 [==============================] - 0s 7ms/step - loss: 1173675.1250
Epoch 10/100
58/58 [==============================] - 0s 6ms/step - loss: 1171833.1250
Epoch 11/100
58/58 [==============================] - 0s 6ms/step - loss: 1170014.1250
Epoch 12/100
58/58 [==============================] 